### 휴먼지능정보공학과 201910787 김영서
7주: zscore, cdf 계산<br>
성적데이터는 n이 적지만, 정규분포를 이룬다고 가정하자.

In [74]:
import sys
import os

home=os.path.expanduser("~")
os.environ["PYSPARK_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYSPARK_DRIVER_PYTHON"]="C:\\Users\\zaqxs\\Anaconda2\\envs\\py35\\python.exe"
os.environ["PYLIB"]=os.path.join(os.environ["SPARK_HOME"],'python','lib')
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'py4j-0.10.9-src.zip'))
sys.path.insert(0,os.path.join(os.environ["PYLIB"],'pyspark.zip'))

In [75]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

##  1-1 성적데이터로 DataFrame을 생성

In [76]:
marks=[
    "김하나, English, 100",
    "김하나, Math, 80",
    "임하나, English, 70",
    "임하나, Math, 100",
    "김갑돌, English, 82.3",
    "김갑돌, Math, 98.5"
]

#### RDD 생성

In [77]:
_marksRdd=spark.sparkContext.parallelize(marks).map(lambda x:x.split(','))

#### DataFrame 생성

In [78]:
_marksDf=spark.createDataFrame(_marksRdd, schema=["name", "subject", "mark"])

In [79]:
_marksRdd.take(1)

[['김하나', ' English', ' 100']]

In [80]:
_marksDf.printSchema()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: string (nullable = true)



#### 데이터 타입 변경

In [81]:
from pyspark.sql.types import FloatType

_marksDf = _marksDf.withColumn('markF', _marksDf['mark'].cast(FloatType()))

In [82]:
_marksDf.printSchema()

root
 |-- name: string (nullable = true)
 |-- subject: string (nullable = true)
 |-- mark: string (nullable = true)
 |-- markF: float (nullable = true)



In [83]:
_marksDf.show()

+------+--------+-----+-----+
|  name| subject| mark|markF|
+------+--------+-----+-----+
|김하나| English|  100|100.0|
|김하나|    Math|   80| 80.0|
|임하나| English|   70| 70.0|
|임하나|    Math|  100|100.0|
|김갑돌| English| 82.3| 82.3|
|김갑돌|    Math| 98.5| 98.5|
+------+--------+-----+-----+



## 1-2 zscore 컬럼을 생성
zscore를 계산하려면, 평균과 표준편차를 알아야 한다.<BR>
계산식에 F함수를 직접 사용하면 오류가 발생한다. 따로 평균과 표준편차를 구해서 계산식에서 사용해야 한다.<BR>

In [94]:
from pyspark.sql import functions as F

_markStats = _marksDf.select(
    F.mean('markF').alias('mean'),  # alias: 이름 변경
    F.stddev('markF').alias('std')
).collect()

In [95]:
_markStats

[Row(mean=88.46666717529297, std=12.786190172956093)]

In [96]:
meanMark = _markStats[0]['mean']
stdMark = _markStats[0]['std']

In [97]:
meanMark

88.46666717529297

In [98]:
stdMark

12.786190172956093

#### Float type으로 형변환

In [99]:
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType

zscoreUdf = F.udf(lambda x: (x-meanMark)/stdMark, FloatType())

In [100]:
_marksDf=_marksDf.withColumn("zscore", zscoreUdf(_marksDf['markF']))

In [101]:
_marksDf.show()

+------+--------+-----+-----+-----------+
|  name| subject| mark|markF|     zscore|
+------+--------+-----+-----+-----------+
|김하나| English|  100|100.0|  0.9020148|
|김하나|    Math|   80| 80.0| -0.6621728|
|임하나| English|   70| 70.0| -1.4442666|
|임하나|    Math|  100|100.0|  0.9020148|
|김갑돌| English| 82.3| 82.3|-0.48229098|
|김갑돌|    Math| 98.5| 98.5| 0.78470075|
+------+--------+-----+-----+-----------+



## 1-3 cdf 컬럼을 생성
scipy.stats.norm.cdf() 함수는 데이터타입을 float로 맞추어 주어야 한다.

cdf는 평균=0, 표준편차=1을 기본 값으로 누적확률을 계산한다.

In [111]:
from scipy.stats import norm
from pyspark.sql.functions import udf

In [112]:
cdf_udf=udf(lambda x: float(norm.cdf(x)))

In [113]:
_marksDf=_marksDf.withColumn("cdf", cdf_udf(_marksDf['zscore']))

In [114]:
_marksDf.show()

+------+--------+-----+-----+-----------+-------------------+
|  name| subject| mark|markF|     zscore|                cdf|
+------+--------+-----+-----+-----------+-------------------+
|김하나| English|  100|100.0|  0.9020148| 0.8164754981807292|
|김하나|    Math|   80| 80.0| -0.6621728| 0.2539302463290559|
|임하나| English|   70| 70.0| -1.4442666| 0.0743320011235712|
|임하나|    Math|  100|100.0|  0.9020148| 0.8164754981807292|
|김갑돌| English| 82.3| 82.3|-0.48229098|0.31479962882028223|
|김갑돌|    Math| 98.5| 98.5| 0.78470075| 0.7836854740814176|
+------+--------+-----+-----+-----------+-------------------+

